In [1]:
import sys
sys.path.append('../../lib')
import numpy as np
import libry as ry
import time

In [2]:
C = ry.Config()
C.view()
C.addFile('../Control/minimal/scene.g')

In [3]:
tau=.01

In [4]:
CS = ry.CtrlSet()

CS.add_qControlObjective(2, 1e-2*np.math.sqrt(tau), C)
CS.add_qControlObjective(1, 1e-1*np.math.sqrt(tau), C)
pos = CS.addObjective(C.feature(ry.FS.poseDiff, ["gripper", "target"], [1e0]), ry.OT.sos, .1)
CS.addObjective(C.feature(ry.FS.accumulatedCollisions, ["ALL"], [1e2]), ry.OT.eq)

In [5]:
ctrl = ry.CtrlSolver(C, tau, 2)

In [6]:
for t in range(0,100):
    ctrl.set(CS)
    ctrl.update(C)
    q = ctrl.solve(C)
    C.setJointState(q)
    C.computeCollisions()

#    ctrl.report();
 #   C.watch(false, STRING("t:" <<t));
    time.sleep(.01)
#    if(pos->status>AS_running) break;
#//    if(CS.isConverged(ctrl.komo.pathConfig)) break;

In [ ]:
import os
os._exit(0)

In [ ]:
import sys
sys.path.append('../../lib')
import numpy as np
import libry as ry
import time

In [ ]:
C = ry.Config()
C.addFile('../Control/minimal/pandas.g')
C.view()
tau=.01

In [10]:
approach = ry.CtrlSet()
approach.addObjective(C.feature(ry.FS.vectorZDiff, ["object", "R_gripperCenter"], [1e1]), ry.OT.sos, .005)
approach.addObjective(C.feature(ry.FS.positionRel, ["object", "R_gripperCenter"], [1e1], [.0, 0., -.15]), ry.OT.sos, .005)
#approach.symbolicCommands.append({"openGripper", "R_gripper"});

preGrasp = ry.CtrlSet()
#immediate constraint:
preGrasp.addObjective(C.feature(ry.FS.insideBox, ["object", "R_gripperPregrasp"], [1e0]), ry.OT.ineq, -1)
#transient:
preGrasp.addObjective(C.feature(ry.FS.vectorZDiff, ["object", "R_gripperCenter"], [1e1]), ry.OT.sos, .005)
preGrasp.addObjective(C.feature(ry.FS.positionDiff, ["R_gripperCenter", "object"], [1e1]), ry.OT.sos, .002)
#preGrasp.symbolicCommands.append({"preOpenGripper", "R_gripper"});

grasp = ry.CtrlSet()
grasp.addObjective(C.feature(ry.FS.vectorZ, ["R_gripperCenter"], [], [0., 0., 1.]), ry.OT.eq, -1.)
grasp.addObjective(C.feature(ry.FS.positionDiff, ["R_gripperCenter", "object"], [1e1]), ry.OT.eq, -1.)
#grasp.symbolicCommands.append({"closeGripper", "R_gripper"});

controls = ry.CtrlSet()
controls.add_qControlObjective(2, 1e-3*np.math.sqrt(tau), C)
controls.add_qControlObjective(1, 1e-1*np.math.sqrt(tau), C)
#auto coll = ctrl.addObjective(FS_accumulatedCollisions, {}, OT_eq, {1e2});

RuntimeError: /home/jason/git/thesis_2020/rai/rai/Kin/kin.cpp:getFrameIDs:371(-2) frame name 'object' doesn't exist

In [ ]:
ctrl = ry.CtrlSolver(C, tau, 2)

In [ ]:
for t in range (0,100):

    if grasp.isConverged(ctrl):
        break;
    elif grasp.canBeInitiated(ctrl):
        ctrl.set(grasp)
    elif preGrasp.canBeInitiated(ctrl):
        ctrl.set(preGrasp)
    elif approach.canBeInitiated(ctrl):
        ctrl.set(approach)

    ctrl.update(C)
    q = ctrl.solve()
    C.setJointState(q)
    C.computeCollisions()

#     ctrl.report();
#     C.watch(false, STRING(txt <<"t:" <<t));
    time.sleep(.01);
# //    if(c_pos->status>AS_running) break;